In [1]:
import arbor
from arbor import mpoint
from arbor import mnpos
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook

In [2]:
tree = arbor.segment_tree()
# Start with a cylinder segment for the soma (with tag 1)
tree.append(mnpos, mpoint(0,   0.0, 0, 2.0), mpoint( 4,  0.0, 0, 2.0), tag=1)
# Construct the first section of the dendritic tree,
# comprised of segments 1 and 2, attached to soma segment 0.
tree.append(0,     mpoint(4,   0.0, 0, 0.8), mpoint( 8,  0.0, 0, 0.8), tag=3)
tree.append(1,     mpoint(8,   0.0, 0, 0.8), mpoint(12, -0.5, 0, 0.8), tag=3)
# Construct the rest of the dendritic tree.
tree.append(2,     mpoint(12, -0.5, 0, 0.8), mpoint(20,  4.0, 0, 0.4), tag=3)
tree.append(3,     mpoint(20,  4.0, 0, 0.4), mpoint(26,  6.0, 0, 0.2), tag=3)
tree.append(2,     mpoint(12, -0.5, 0, 0.5), mpoint(19, -3.0, 0, 0.5), tag=3)
tree.append(5,     mpoint(19, -3.0, 0, 0.5), mpoint(24, -7.0, 0, 0.2), tag=3)
tree.append(5,     mpoint(19, -3.0, 0, 0.5), mpoint(23, -1.0, 0, 0.2), tag=3)
tree.append(7,     mpoint(23, -1.0, 0, 0.2), mpoint(26, -2.0, 0, 0.2), tag=3)
# Two segments that define the axon, with the first at the root, where its proximal
# end will be connected with the proximal end of the soma segment.
tree.append(mnpos, mpoint(0,   0.0, 0, 2.0), mpoint(-7,  0.0, 0, 0.4), tag=2)
tree.append(9,     mpoint(-7,  0.0, 0, 0.4), mpoint(-10, 0.0, 0, 0.4), tag=2)
#
# Get the morphology
morph = arbor.morphology(tree)
#
# Create the cable cell
labels = arbor.label_dict( {"soma": "(tag 1)", "axon": "(tag 2)", "dend": "(tag 3)",
                            "stim": "(location 0 0.5)", "everywhere": "(all)"} )
cell = arbor.cable_cell(tree, labels)

In [3]:
# Set global properties
cell.set_properties(Vm=-70, cm=1.0, rL=500.) # tempK=30+273.5
cell.set_ion(ion="na", rev_pot=50.)
cell.set_ion(ion="k", rev_pot=-77.)
# Set mechanisms
#leaky_chnl = arbor.mechanism('passive/el=-54.3', {'g': 0.0003})
#leaky_chnl = arbor.mechanism("pass", {'e': -54.3, 'g': 0.0003})
leaky_chnl = arbor.mechanism('pas', {'e': -54.3, 'g': 0.0003})
#leaky_chnl = arbor.mechanism('passive/e=-54.3', {'g': 0.0003})
hh_na_k_chnl = arbor.mechanism("hh", {"gnabar": 0.120,
                                      "gkbar": 0.036,
                                      "gl": 0.})
# Attach mechanism and paint ion parameters
all_regions = [' (region "soma") ', ' (region "dend") ', ' (region "axon") ']
cell.paint("(join"+" ".join( (all_regions) )+")", "na", rev_pot=50)
cell.paint('(join (region "soma") (region "dend") (region "axon"))', "k", rev_pot=-77)
cell.paint('(join (region "soma") (region "dend") (region "axon"))', hh_na_k_chnl)
cell.paint('"everywhere"', leaky_chnl)
#cell.paint('"soma"', "na", rev_pot=50)
#cell.paint('"soma"', "k", rev_pot=-77)
#cell.paint('"soma"', hh_na_k_chnl)
#
#cell.paint('"axon"', "na", rev_pot=50)
#cell.paint('"axon"', "k", rev_pot=-77)
#cell.paint('"axon"', hh_na_k_chnl)
#
#cell.paint('"dend"', "na", rev_pot=50)
#cell.paint('"dend"', "k", rev_pot=-77)
#cell.paint('"dend"', hh_na_k_chnl)
#
#cell.paint('"everywhere"', leaky_chnl)
# Set stimulus
cell.place('"stim"', arbor.iclamp(10, 2, 0.8))
cell.place('"stim"', arbor.iclamp(50, 2, 0.8))
cell.place('"stim"', arbor.iclamp(80, 2, 0.8))
# Add spike detector
cell.place('"stim"', arbor.spike_detector(-10)) # -10mV threshold
# Improve discretization precision to 2 um
cell.compartments_length(2)

# Objectives:
1. **Wrap the cable cell into an Arbor single cell model**
2. **Add probe for recording**
3. **Run the simulation**
4. **Plot**

# 1. Wrap the created (already setup) cable cell

In [4]:
cell_model = arbor.single_cell_model(cell)

# 2. Add probe for recording

In [5]:
cell_model.probe("voltage", '"stim"', 10000) # another kind of probe is "current" (new)

# 3. Run the simulation

In [6]:
cell_model.run(100) # 100ms

# 4. Plot

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
#output_file("arbor_mc_demo.html")
graph = figure(title = "Arbor MC Demo") 
graph.line(cell_model.traces[0].time, cell_model.traces[0].value)
show(graph)

In [11]:
help(cell)

Help on cable_cell in module arbor._arbor object:

class cable_cell(pybind11_builtins.pybind11_object)
 |  Represents morphologically-detailed cell models, with morphology represented as a
 |  tree of one-dimensional cable segments.
 |  
 |  Method resolution order:
 |      cable_cell
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(...)
 |      __init__(*args, **kwargs)
 |      Overloaded function.
 |      
 |      1. __init__(self: arbor._arbor.cable_cell, morphology: arb::morphology, labels: arbor._arbor.label_dict) -> None
 |      
 |      2. __init__(self: arbor._arbor.cable_cell, segment_tree: arb::segment_tree, labels: arbor._arbor.label_dict) -> None
 |      
 |      Construct with a morphology derived from a segment tree.
 |  
 |  __repr__(...)
 |      __repr__(self: arbor._arbor.cable_cell) -> str
 |  
 |  __str__(...)
 |      __str__(self: arbor._arbor.cable_cell) -> str
 |  
 |  cables(...)
 |      cables(sel

In [8]:
!pip3 install git+https://github.com/arbor-sim/arbor.git

  Cloning https://github.com/arbor-sim/arbor.git to /tmp/pip-req-build-8pmbj3nb
  Created wheel for arbor: filename=arbor-0.4.1.dev0-cp38-cp38-linux_x86_64.whl size=1250518 sha256=c5ae65fae8d8d394234c140dacfd85dd73700b79cea5ab84398dc6d02257bf0a
  Stored in directory: /tmp/pip-ephem-wheel-cache-n20ts23z/wheels/e8/ca/6a/b938b0965e8593a709ac3d077ea761e6caedb18dab950e7e90
Successfully built arbor
  Attempting uninstall: arbor
    Found existing installation: arbor 0.3.1.dev0
    Uninstalling arbor-0.3.1.dev0:
      Successfully uninstalled arbor-0.3.1.dev0
You should consider upgrading via the '/envs/py3-pyNNarbor/bin/python -m pip install --upgrade pip' command.


In [7]:
!pip show arbor

Name: arbor
Version: 0.4.1.dev0
Summary: High performance simulation of networks of multicompartment neurons.
Home-page: https://github.com/arbor-sim/arbor
Author: The Arbor dev team.
Author-email: None
License: UNKNOWN
Location: /envs/py3-pyNNarbor/lib/python3.8/site-packages
Requires: 
Required-by: 


In [ ]:
cell.